The next cell import all the libraries and the other folder that we will need

In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 


initialize the initial grid and position

In [ ]:
#start , goal , grid = "init_map"
start_pos = (0,0)
goal = (43,33)
map_lenght = 5
nb_of_square_by_side = 50

Next cell is to create an instance Robot and Map 
  

In [ ]:
George = Robot(start_pos, goal)
Lausanne = Map(map_lenght, nb_of_square_by_side)

Global control 

--> Optimal path calculation

In [ ]:
# Compute the optimal path and visited nodes
path, visitedNodes = op.path_computation(George.get_start() , George.get_goal() , Lausanne.get_lenght(), Lausanne.get_map())

George.set_path(path) 
George.set_visit_nodes(visitedNodes) 

# Display the optimal path
op.display_map(Lausanne.get_lenght(),  Lausanne.get_map(),  George.get_visit_nodes(), George.get_path(), George.get_start(), George.get_goal())


start threads

In [ ]:
# Threads de l'acquisition d'image et filtre Kalman --> C'est l'ordi qui envoit ça.
# Met à jour la position du robot
# Affiche sur ordi la map analisée

# Thread pour les roues? --> implique class motors

# Main loop

In [3]:
global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)

while True:#(George.get_pos() != George.get_goal()):
    
    
    # Reupere la position et angle --> XAV
    # Current goal update --> ANTOINE
    # Check obstacle --> ALICIA
    # Faire avancer --> NOUR
    
    """
    # Path calculation?
    Geroge.set_current(op.path_update(George.get_pos(), George.get_err_pos(), George.get_path(), George.get_current())) # Update the "local" goal
    current_goal = George.get_path()(Geroge.get_current())
    obstacle = check_opstacle() # Booleen
    if(obstacle):
        manoeuvre_devitement
    else : 
        go_forward_to_the_local_goal(George.get_angle(), George.getpos(), current_goal)
    """
    
    

    
    
    #LOCAL NAVIGATION
    if move:
        #Thymio is avoiding obstacles
        move = avoid_obstacle() 
    else:
        #Thymio is checking if there's an obstacle in front of it
        move = check_cars() 
        #FUNCTION NOUR --> OPTIMAL PATH
    
    sleep(0.5)

#led_arrivé

NameError: name 'prox_horizontal' is not defined